# Clusters Visualization
This notebook is used to be able to choose significant **K**, among all the variuos **K**, by cluster visualization.

In [ ]:
import os
import json
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import sys
sys.path.append(str(Path("../src").resolve()))
from config import *


def load_clustering_data() -> dict:
    '''
    Load clustering data as a dictionary.
    
    Parameters:
        None
    
    Returns:
        dict: The dictionary in the form {k, data}
    '''
    # get all the json files that have samples (CLUSTERING_FILANEAMEi.json)
    json_files = list(Path(os.path.join("..", CLUSTERING_DIRECTORY)).glob(f"{CLUSTERING_FILENAME}*.json"))

    # check if the json is valid (to do)...

    def get_k_from_json(json_file):
        # get k value from a json
        with open(json_file, "r") as f:
            data = json.load(f)
        return data.get("k", 0)

    # sort for k
    json_files = sorted(json_files, key=get_k_from_json)

    clustering_data = {}
    for json_file in json_files:
        with open(json_file, "r") as f:
            data = json.load(f)
        clustering_data[data['k']] = data

    return clustering_data

def find_image_path(image_name: str) -> str | None:
    '''
    Find the complete path given the image name.
    
    Parameters:
        image_name (str): The name of the image
    
    Returns:
        str o None: Complete path if found, None otherwise
    '''
    for ext in IMAGES_EXTENSIONS:
        for candidate_ext in [ext.lower(), ext.upper()]:
            candidate_path = os.path.join("../", IMAGES_DIRECTORY, image_name + candidate_ext)
            if os.path.exists(candidate_path):
                return candidate_path
    return None

# ==============================================================

def show_scores(inertia_score: float, silhouette_score: float):
    print(f"Inertia Score: {inertia_score}")
    print(f"Silhouette Score: {silhouette_score}\n")


def show_cluster_count(cluster_counts: list):

    if not all(isinstance(x, int) for x in cluster_counts):
        raise ValueError("cluster_counts has to be a list of integers")
    
    if len(cluster_counts) < MIN_N_CLUSTERS or len(cluster_counts) > MAX_N_CLUSTERS:
        raise ValueError(f"Not valid cluster_counts: it doesn't respect MIN_N_CLSUTER={MIN_N_CLUSTERS} and MAX_N_CLUSTERS={MAX_N_CLUSTERS}")

    cluster_ids = list(range(len(cluster_counts)))

    min_width, max_width = 6, 18
    
    n_clusters = len(cluster_counts)
    width = min_width + (max_width - min_width) * (n_clusters - MIN_N_CLUSTERS) / (MAX_N_CLUSTERS - MIN_N_CLUSTERS)
    height = 4

    plt.figure(figsize=(width, height))

    plt.bar(cluster_ids, cluster_counts, color="skyblue", edgecolor="black")

    plt.xticks(cluster_ids)

    plt.xlabel("Cluster ID")
    plt.ylabel("Number of images")
    plt.title("Distribution of images in clusters")
    
    for i, count in enumerate(cluster_counts):
        plt.text(i, count + 0.05, str(count), ha="center", va="bottom")

    plt.tight_layout()
    plt.show()


def show_centroid_and_samples(k: int, cluster_id: int, cluster_centroid: str, cluster_samples: list, max_per_row=5, save=False) -> None:
    '''
    If save = False, shows the centroid and the samples as an image in the notebook, otherwise save the image as a .png.

    Parameters:
    k (int): The number of cluster
    cluster_id (int): The cluster id among all the clusters
    cluster_centroid (str): The name of the image that is the centroid
    cluster_samples (list): The list of image's names that are the samples
    max_per_row (int): The amount of images to show for each row
    save (bool): If the function has to save or not the image

    Returns:
    None
    '''
    if k < MIN_N_CLUSTERS or k > MAX_N_CLUSTERS:
        raise ValueError(f"k has to be in the interval [{MIN_N_CLUSTERS}, {MAX_N_CLUSTERS}]")
    
    if cluster_id < 0 or cluster_id >= k:
        raise ValueError(f"cluster_id has to be in the interval [0, {k - 1}]")

    if not all(isinstance(x, str) for x in cluster_samples):
        raise ValueError("cluster_samples has to be a list of names")
    
    if max_per_row < 1:
        raise ValueError("max_per_row has to be greater than 0")

    n_images = len(cluster_samples)
    n_cols = min(max_per_row, n_images)
    n_rows = (n_images + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*3, n_rows*3 + 1))
    axes = axes.flatten() if n_images > 1 else [axes]

    # initial info to identify cluster
    fig.suptitle(f"K = {k}, Cluster ID = {cluster_id}", fontsize=12, y=1.0) 

    for ax, img_name in zip(axes, cluster_samples):
        image_path = find_image_path(img_name)
        if image_path:
            img = Image.open(image_path)
            ax.imshow(img)
        else:
            ax.text(0.5, 0.5, f"File {img_name} not found", ha='center', va='center', color='red')
        ax.set_title(img_name, fontsize=8)
        ax.axis('off')

    for ax in axes[len(cluster_samples):]:
        ax.axis('off')
    plt.tight_layout()

    if save:
        plt.savefig(os.path.join("../", CLUSTER_IMAGES_DIRECTORY, f"{CLUSTER_IMAGE_FILENAME}{k}_{cluster_id}.png"), bbox_inches='tight')
        plt.close()
    else:
        plt.show()

# ==============================================================


def on_save_button_click(b):
    cluster_id = cluster_widget.value
    with output_save:
        clear_output()  # clear previous messages
        if cluster_id == 'Select':
            print("Select a valid cluster")
            return

        data = clustering_data[k_widget.value]
        cluster_centroid = data["centroids"][cluster_id]
        cluster_samples = data["samples"][cluster_id]

        # save the image
        show_centroid_and_samples(k_widget.value, cluster_id, cluster_centroid, cluster_samples, save=True)
        print("Saved")


# ==============================================================

# load all json in a dictionary {k, data}
clustering_data = load_clustering_data()


# k selection
k_widget = widgets.Dropdown(
    options=sorted(clustering_data.keys()),
    description='Select K:'
)

# cluster id selection
cluster_widget = widgets.Dropdown(
    options=['Select'],
    value='Select',
    description='Cluster ID'
)

# save button
save_button = widgets.Button(
    description="Save Cluster",
    tooltip="Save cluster as an image"
)


# output to show results
output_k = widgets.Output()
output_cluster = widgets.Output()
output_save = widgets.Output()


def on_k_change(change):
    if change['name'] == 'value':

        k = change['new']

        # update output of first widget
        with output_k:
            clear_output()
            print(f"You selected K = {k}\n")

            data = clustering_data[k]
            inertia_score = data.get("inertia_score")
            silhouette_score = data.get("silhouette_score")
            cluster_counts = data.get("cluster_counts")

            show_scores(inertia_score, silhouette_score)
            show_cluster_count(cluster_counts)
        
        # reset second widget to 'Select'
        new_options = ['Select'] + list(range(k))
        cluster_widget.options = new_options
        cluster_widget.value = 'Select'

        # reset output_save
        with output_save:
            clear_output()


def on_action_change(change):
    if change['name'] == 'value':

        cluster_id = change['new']

        with output_cluster:
            clear_output()
            if change['new'] != 'Select':
                print(f"You selected Cluster ID = {cluster_id}\n")

                data = clustering_data[k_widget.value]
                centroids = data["centroids"]
                samples = data["samples"]
                
                show_centroid_and_samples(k_widget.value, cluster_id, centroids[cluster_id], samples[cluster_id])

        # reset output_save
        with output_save:
            clear_output()


# connect functions to widgets
k_widget.observe(on_k_change)
cluster_widget.observe(on_action_change)
save_button.on_click(on_save_button_click)

# visualize all
display(k_widget, output_k, cluster_widget, save_button, output_save, output_cluster)

# initialize output first widget
on_k_change({'name': 'value', 'new': k_widget.value})
